# Bootstrap Calibrations

In [47]:
%matplotlib inline

In [1]:
# imports
from desispec import bootcalib as desiboot
import numpy as np
from astropy.modeling import models, fitting
from xastropy.xutils import afits as xafits

## Find peaks

In [2]:
# Read flat
flat = fits.open('/Users/xavier/DESI/Wavelengths/pix-b0-00000001.fits')[0].data

In [3]:
reload(desiboot)
xpk, ypos, cut = desiboot.find_fiber_peaks(flat)#,debug=True)

Found 500 fibers
Found 20 bundles


In [52]:
# Check
plt.clf()
plt.figure(figsize=(16,7))
xplt = np.arange(cut.size)
plt.plot(xplt,cut, 'k-')
plt.plot(xpk, cut[xpk],'go')
plt.xlim(100,500)
plt.show()

## Trace the flat spectra

### Crude estimate (flux weighted centroid)

In [4]:
reload(desiboot)
xset, xerr = desiboot.trace_crude_init(flat,xpk[0:50],ypos)

### Polynomial fits (test)

In [13]:
yval = np.arange(4096)
ii=4
xval = xset[:,ii]
gdval = xerr[:,ii] < 999.
dfit0 = xafits.func_fit(yval[gdval],xval[gdval],'legendre',6)

In [14]:
fitv = xafits.func_val(yval,dfit0)

In [15]:
# Fit
xdb.xplot(yval,fitv, xtwo=yval,ytwo=xset[:,ii],mtwo='+')

In [16]:
# Residuals
xdb.xplot(yval[gdval],fitv[gdval]-xval[gdval], scatter=True)

### Polynomial fits (True)

In [5]:
reload(desiboot)
xfit, fits = desiboot.fit_traces(xset,xerr)#[:,0:5],xerr[:,0:5])

### QA

In [19]:
reload(desiboot)
desiboot.fiber_trace_qa(flat,xfit)

Writing fiber_trace_qa.pdf QA for fiber trace


## PSF

### Sigma for each fiber (initial guess)

In [6]:
reload(desiboot)
gauss = desiboot.fiber_gauss(flat,xfit,xerr)#,debug=True)

/Users/xavier/DESI/desispec/py/desispec/bootcalib.py:78: RuntimeWarning: invalid value encountered in log
  np.log(amp)-np.log(fnimg) )
/Users/xavier/DESI/desispec/py/desispec/bootcalib.py:78: RuntimeWarning: invalid value encountered in sqrt
  np.log(amp)-np.log(fnimg) )
/Users/xavier/DESI/desispec/py/desispec/bootcalib.py:78: RuntimeWarning: divide by zero encountered in true_divide
  np.log(amp)-np.log(fnimg) )


### Fit a 2nd Order Polynomial

In [7]:
fiber = np.arange(gauss.size)
fdict,mask = xafits.iter_fit(fiber, gauss, 'polynomial', 2)

In [8]:
fdict

{'coeff': array([  1.13885772e+00,  -6.22223052e-03,  -7.41988283e-05]),
 'func': 'polynomial',
 'order': 2,
 'xmax': 49,
 'xmin': 0}

In [ ]:
plt.clf()
plt.scatter(fiber,gauss)
plt.plot(fiber, xafits.fit_val()
#plt.xlim(100,500)
plt.show()